In [4]:
import os
import sys
parent_dir = os.path.abspath(os.path.join(os.getcwd(), '..'))
sys.path.insert(0, parent_dir)

parent_dir = os.path.abspath(os.path.join(os.getcwd().split('render')[0],'gaussian_data'))
sys.path.insert(0, parent_dir)

from gaussian_data.Camera import Camera
from gaussian_data.Frame import Frame
import gaussian_data.Plotters
import gaussian_data.Utils as Utils

from GaussianSplat import GaussianSplat


import numpy as np
from plyfile import PlyData
import numpy as np
import argparse
from io import BytesIO
import matplotlib.pyplot as plt
import Plotters
import plotly
import plotly.graph_objects as go
import scipy.io
import pandas as pd
from Render import Render

%matplotlib qt

# load hull
path = 'G:/My Drive/Research/gs_data/mov19_2022_03_03/'
real_coord = scipy.io.loadmat(f'{path}/3d_pts/real_coord.mat')['all_coords']
points_3d = {body_wing : pd.DataFrame(Utils.load_hull(body_wing,path),columns = ['X','Y','Z','frame']) for body_wing in ['body','rwing','lwing']}
# initilize objects
frames = [1520]#list(range(500,700,13))

image_names,points_in_idx = Utils.define_frames(frames,points_3d)
cameras = {f'cam{cam + 1}':Camera(path,cam) for cam in range(4)}
frames = {f'{im_name}.jpg':Frame(path,im_name,points_in_idx[im_name.split('CAM')[0]],real_coord,idx) for idx,im_name in enumerate(image_names)}
# map 3d voxels to 2d pixels
[frames[im_name].map_3d_2d(croped_image = True) for im_name in frames.keys()]
voxel_dict,colors_dict = Utils.get_dict_for_points3d(frames)


In [26]:
# load gaussian splatting output
input_file = "G:/My Drive/Research/gs_data/mov19_2022_03_03/output/scaling_lr0.005_percent_dense0.001_densify_grad_threshold0.0002_position_lr_init1.6e-07/point_cloud/iteration_30000/point_cloud.ply"
input_file = "D:\Documents/2dgs_output/point_cloud3.ply"


gs = GaussianSplat(input_file)  

# reproject and filter background
gs_filtered = gs.filter(gs.projection_filter(frames,gs.xyz,croped_image = True),path = gs.path)
gs_filtered.save_gs()




ValueError: no field of name scale_2

In [24]:
plt.plot(rgb[rgb[:,0]<1])

In [ ]:
import sh_utils
input_file = "D:\Documents/2dgs_output/point_cloud.ply"

vertices = PlyData.read(input_file)["vertex"] 
sh = np.column_stack([vertices[key] for key in vertices.data.dtype.names if 'rest' in key or 'dc' in key])
rgb = sh_utils.rgb_from_sh(0,sh,xyz = None,camera_position = None)
xyz = np.column_stack((vertices["x"], vertices["y"], vertices["z"]))
fig = go.Figure()
Plotters.scatter3d(fig,xyz[rgb[:,0]<1],rgb[rgb[:,0]<1],3)
fig.show()

plt.plot(vertices['nx'])

memmap([0., 0., 0., ..., 0., 0., 0.], dtype=float32)

In [45]:
plt.plot(vertices['ny'])

In [41]:
import open3d as o3d
mesh = o3d.io.read_point_cloud(input_file) # Read the point cloud

# Visualize the point cloud within open3d

# Visualize the point cloud with normals
mesh.normals = np.column_stack((np.array(vertices['nx']),np.array(vertices['ny']),np.array(vertices['nz'])))
o3d.visualization.draw_geometries([mesh], point_show_normal=True)

TypeError: (): incompatible function arguments. The following argument types are supported:
    1. (self: open3d.cpu.pybind.geometry.PointCloud, arg0: open3d.cpu.pybind.utility.Vector3dVector) -> None

Invoked with: PointCloud with 39211 points., array([[0., 0., 0.],
       [0., 0., 0.],
       [0., 0., 0.],
       ...,
       [0., 0., 0.],
       [0., 0., 0.],
       [0., 0., 0.]], dtype=float32)

In [38]:
plt.plot(np.asarray(mesh.normals))

In [3]:
# render image 
image_name = f'{image_names[0]}.jpg'

cam1 = frames[image_name].get_croped_camera()
cam4 = frames[f'{image_names[0]}.jpg'].get_croped_camera()

rotmat = np.array([[1,0,0],[0,1,0],[0,0,1]])

K_crop = cam4.K
K_crop[0,0] = K_crop[0,0]
K_crop[1,1] = K_crop[1,1]

R = rotmat @ cam4.R
X0 = cam4.X0 + np.array([[0,0,0]]).T
cam = Camera(cam1.path,5,cam = {'camera': np.hstack((K_crop,R,X0))[:, :, np.newaxis]},image_size = [160,160])

cam = frames[f'{image_names[0]}.jpg'].get_croped_camera()



render = Render(gs,cam,tiles = [1,10],block_xy = [16,16], image_size = [160,160])
image = render.render_image()
plt.figure(),plt.imshow(image)




Clipping input data to the valid range for imshow with RGB data ([0..1] for floats or [0..255] for integers).


(<Figure size 640x480 with 1 Axes>,
 <matplotlib.image.AxesImage at 0x2bdbbc6b220>)

## Elliptical Gaussian (from EWA article) - Math explained


we define an elliptical gaussian: 

Equation 1: $G_\mathbf{V}(\mathbf{x} - \mathbf{p}) = \frac{1}{2 \pi |\mathbf{V}|} \frac{1}{2} e^{-\frac{1}{2} (\mathbf{x} - \mathbf{p})^T \mathbf{V}^{-1} (\mathbf{x} - \mathbf{p})}.$

* $V$ is the covariance matrix
* $\mathbf{x}$ - a 3d coordinate (in gaussian axes)
* $\mathbf{p}$ - the mean of the gaussian 
### Affine projection of the gauusian
We define an arbitrary affine mapping from object space $ \mathbf{u} = \Phi(\mathbf{x}) $, where $ \Phi(\mathbf{x}) = \mathbf{Mx + c} $, 

and we substitute $ \mathbf{x} = \Phi^{-1}(\mathbf{u}) $. Since $ \Phi^{-1}(\mathbf{u}) = \mathbf{M}^{-1}(\mathbf{u} - \mathbf{c}) $, we get:

$$
G_\mathbf{V}(\Phi^{-1}(\mathbf{u}) - \mathbf{p}) = 
\frac{1}{|M^{-1}|} \cdot \frac{1}{(2 \pi)^{3/2} |\mathbf{V}|^{1/2}} e^{-\frac{1}{2} (\mathbf{M}^{-1}(\mathbf{u} - \mathbf{c}) - \mathbf{p})^T \mathbf{V}^{-1} (\mathbf{M}^{-1}(\mathbf{u} - \mathbf{c}) - \mathbf{p})} = 
$$

$$
\frac{1}{|M^{-1}|} \cdot \frac{1}{(2 \pi)^{3/2} |\mathbf{V}|^{1/2}} e^{-\frac{1}{2} (\mathbf{u} - \mathbf{c} - \mathbf{Mp})^T \mathbf{M}^{-1} \mathbf{V}^{-1} (\mathbf{M}^{-1})^T (\mathbf{u} - \mathbf{c} - \mathbf{Mp})} = 
$$

$$
\frac{1}{|M^{-1}|} \cdot \frac{1}{(2 \pi)^{3/2} |\mathbf{V}|^{1/2}} e^{-\frac{1}{2} (\mathbf{u} - \Phi(\mathbf{p}))^T (\mathbf{MVM}^T)^{-1} (\mathbf{u} - \Phi(\mathbf{p}))}.
$$

from this, to ease the writing we define equation 2: 

Equation 2: $G \mathbf{V}(\Phi^{-1}(u) - \mathbf{p}) = \frac{1}{|\mathbf{M}^{-1}|} G_{ \mathbf{M} \mathbf{V} \mathbf{M}^T} (u - \Phi(\mathbf{p})).$

#### The covariance matrix
$V$, the covariance matrix is a symetrix matrix that can be ragarded as a transformation. it scales and shears the data (thus defining the orientation and ratio between the gaussian principal axes). 
As mentioned above, the gaussian is transformed using an affine mapping $\Phi(X)$. Usually we use this kind of mapping to transform a vector or a 3D point to a new FoR (change basis). Because the covariance matrix is a transformation, in order to change its FoR (basis), we will perform $\mathbf{V_{1}} = MVM^{T}$. We first multiply $VM^{T}$  - transform (whatever xyz we multiply) from camera to body ($M^{T}$) and then multiply by $V$ which is in object coordinates - now we have coordinates in object basis after shear and scale. we then multiply by $M$ again to transform back to camera coordinates. 


### Projection


![alt text](ray_ewa.png)
#### Ray space

We converted to camera space, now we want to convert to ray space. The coordinates in ray space are defined such that $x_0, x_1$ are the coordinates on the image plane and the devision by $u_2$ adds prespective. $u_3$ is then defined as the euclidean distance from the point to the origin (camera). 


$$ \text{ray space to camera: }
\begin{pmatrix}
x_0 \\
x_1 \\
x_2 \\

\end{pmatrix} = m(u) = \begin{pmatrix}
u_0 / u_2 \\
u_1/u_2 \\
||(u_0, u_1, u_2)||^{T} \\
\end{pmatrix}

\text{,camera to ray space: }
\begin{pmatrix}
u_0 \\
u_1 \\
u_2 \\

\end{pmatrix} = m^{-1}(x) = \begin{pmatrix}
x_0/||(x_0,x_1,1)^T|| \cdot x_2 \\
x_1/||(x_0,x_1,1)^T|| \cdot x_2 \\
1/||(x_0,x_1,1)^T|| \cdot x_2 \\

\end{pmatrix}


$$
  
Ray space is defined with 3 coordinates $(x_0,x_1,x_2)$ where $x_0$ and $x_1$ represents the location of the pixel and $x_3$ is the euclidean distance from the camera origin to $(x_1,x_2). This transformation is not affine due to the devision by $u_2$, parallel lines are not preserved and the shape of the gaussian will be distorted. (in addition, the dimention is problematic).
In order to solve this a local affine approximation $\mathbf{m_{u_k}}$ of the projection transformation is defined. The approximation is the first to terms of the Tayor expansion of $\mathbf{m}$ at point $\mathbf{u}_{k}$:
$$\mathbf{m_{u_k}\mathbf(u)} = \mathbf x_k+\mathbf J_{\mathbf{u}_k}\cdot ( \mathbf u-\mathbf u_k )$$
where $\mathbf x_k=\mathbf m(\mathbf u_k)$ is the mean of the gaussian in ray space (can be transformed using the mapping m, the projection matrix) and the Jacobian $ J_{u_k} = \frac{\partial m}{\partial u}(u_k) $

### The gaussian mapping
#### Object to camera coordinates (as detailed in Affine projection of the gauusian)

The reconstruction kernels are initially given in object space, which has coordinates $t =(t_0,t_1,t_2)^T$ . 
* Gaussian reconstruction kernels in object space: $r"_k (t)=G_{V"_k} (t − t_k)$, where $t_k$ are the voxel positions in object space
* Object coordinates are transformed to camera coordinates using an affine mapping $\mathbf u = \phi(t)$, called viewing transformation. It is defined by a matrix $\mathbf W$ and a translation vector $\mathbf d$ as $\phi(t) = Wt + d$. We transform the reconstruction kernels $G_{V"k} (t − t_k)$ to camera space by substituting $t = \phi ^{−1}(u)$ and using Equation 2 : 

$G \mathbf{V"_{k}}(\Phi^{-1}(u) - t_k) = \frac{1}{|\mathbf{W}^{-1}|} G_{V'_k } (\mathbf u - \mathbf u_k) = r'_k(u\mathbf).$

where $u_k = \phi(\mathbf t_k)$ is the center of the Gaussian in camera coordinates and $r′_k(\mathbf u)$ denotes the reconstruction kernel in camera space. According to equation 2, the covariance matrix in camera coordinates $V′_k$ is given by $V′_k = WV"_kW^T$ (exactly as detailed in Affine projection of the gauusian).

#### Camera to ray space
Reminder: the local affine transformation - $$\mathbf{m_{u_k}\mathbf(u)} = \mathbf x_k+\mathbf J_{\mathbf{u}_k}\cdot ( \mathbf u-\mathbf u_k )$$
$\mathbf{m_{u_k}\mathbf(u)}$ transforms from camera to ray, hence $x_k=m(u_k)$ where $x_k$ is the center of the gaussian in ray space. 
since $\mathbf x = m(\mathbf u)$ we get that $ u = \mathbf m^{-1}(\mathbf(x))$ hence: 
$$ u = \mathbf m^{-1}(x) = \mathbf(x - x_k)\cdot \mathbf J^{-1}_{\mathbf u_k} + \mathbf u_k $$
$$ r_k(\mathbf x) = \frac{1}{|\mathbf W^{-1}|}\cdot G_{\mathbf V'_k}(\mathbf{(x - x_k)\cdot J^{-1}_{\mathbf u_k} + \mathbf u_k - \mathbf u_k}) = \\
\frac{1}{|\mathbf W^{-1}||\mathbf J^{-1}|}G_{\mathbf V'_k}(J^{-1}_{\mathbf u_k}\mathbf{(x - x_k)})\\$$

and the new covariance matrix is defined:  $Vk = JV'_{k}$ $J^T = JWV"_kW^TJ^T$.
where V"_k is the original covariance matrix





### Perspective Projection and Jacobian Derivation

In a pinhole camera model, the perspective projection of a 3D point \((X, Y, Z)\) into 2D image coordinates \((x, y)\) can be expressed as:

$$
\begin{bmatrix}
x \\
y \\
1
\end{bmatrix} = \frac{1}{Z} \cdot
\begin{bmatrix}
f_x \cdot X + c_x \cdot Z \\
f_y \cdot Y + c_y \cdot Z \\
Z
\end{bmatrix}
$$

Where:

* $(x, y)$  : are the projected 2D coordinates on the image plane.
* $(X, Y, Z)$  : are the coordinates in the 3D space. 
* $f_x , f_y$  : are the focal lengths in the x and y directions, respectively. 
* $c_x , c_y$  : are the coordinates of the principal point (optical center) in the image. 

### Derivation of the Jacobian

To derive the Jacobian, we compute the partial derivatives of the projected 2D coordinates \((x, y)\) with respect to the 3D coordinates \((X, Y, Z)\).

1. **From the perspective projection equations**:
   - For \(x\):
   $$
   x = \frac{f_x \cdot X + c_x \cdot Z}{Z}
   $$
   - For \(y\):
   $$
   y = \frac{f_y \cdot Y + c_y \cdot Z}{Z}
   $$

2. **Calculating the derivatives**:
   - The derivatives with respect to \(Z\) yield:
     - For \(x\):
     $$
     \frac{\partial x}{\partial X} = \frac{f_x}{Z}, \quad \frac{\partial x}{\partial Y} = 0, \quad \frac{\partial x}{\partial Z} = -\frac{f_x \cdot X}{Z^2}
     $$
     - For \(y\):
     $$
     \frac{\partial y}{\partial X} = 0, \quad \frac{\partial y}{\partial Y} = \frac{f_y}{Z}, \quad \frac{\partial y}{\partial Z} = -\frac{f_y \cdot Y}{Z^2}
     $$

3. **Forming the Jacobian**:
   Using the derivatives calculated above, the Jacobian matrix \(J\) becomes:
   $$
   J = \begin{bmatrix}
   \frac{f_x}{Z} & 0 & -\frac{f_x \cdot X}{Z^2} \\
   0 & \frac{f_y}{Z} & -\frac{f_y \cdot Y}{Z^2}
   \end{bmatrix}
   $$


### 2D Projection 
We calculate the conics coeffitient for each gaussian : $$Ax^2+Bxy+Cy^2 = 0$$, where $A = \frac{\sigma_y^2}{|V|}$, $B = \frac{\sigma_xy}{|V|}$ and  $C = \frac{\sigma_x^2}{|V|}$
we then multiply by the distace between the gaussiam and the pixel. this is the same as: 
$-\frac{1}{2} (\mathbf{x} - \mathbf{p})^T \mathbf{V}^{-1} (\mathbf{x} - \mathbf{p})$ - the exponent of the gaussian

wIncorporating the scaled conic, which represents the 2D Gaussian as an ellipse in the Gaussian equation, we can express the effect of every Gaussian on the pixel's color as follows:

$$
G_{\mathbf{V}}(\mathbf{x}) = \frac{1}{(2\pi)^{3/2} |\mathbf{V}|^{1/2}} e^{-\frac{1}{2} ( Ax^2+Bxy+Cy^2)}
$$

This equation captures how each Gaussian contributes to the pixel's color based on its distance from the pixel and the shape defined by the covariance matrix.

## tiles
To render the image we devide it to tiles of $ 16 X 16 $, for each tile we sort the gaussians according to the distance from the camera. we then sample all gaussians effect per pixel in the tile (using the power equation to calculate the opacity). We sum the gaussians color values and multiply by the opacity (doing alpha blending, each time we update the opacity according to the already calculated gaussians $color = RGB * \alpha *\alpha_{blend})$ were for eacg gaussian we add we update $\alpha_{blend} =  \alpha_{blend}*(1-\alpha) $ (when we reach 0, its opaque)

